In [1]:
import random

def roll_dice(num_dice):
  """Rolls the specified number of dice and returns a list of results.

  Args:
      num_dice (int): The number of dice to roll (must be greater than 0).

  Returns:
      list[int]: A list containing the results of each die roll (values between 1 and 6).

  Raises:
      ValueError: If the provided number of dice is less than or equal to 0.
  """
  if num_dice <= 0:
    raise ValueError("Number of dice must be greater than 0.")
  return [random.randint(1, 6) for _ in range(num_dice)]

In [2]:
def re_roll(dice, choices):
  """Re-rolls dice based on user choices and returns a new list.

  Args:
      dice (list[int]): The current list of dice values.
      choices (list[str]): A list of strings representing the dice indices to re-roll (e.g., ["1", "3"]).

  Returns:
      list[int]: A new list with the re-rolled dice based on the provided choices.
  """
  new_dice = dice.copy()
  for i in choices:
    new_dice[int(i)-1] = random.randint(1, 6)
  return new_dice

In [3]:
def calculate_score(dice, category):
  """Calculates score based on the chosen category and dice roll.

  Args:
      dice (list[int]): The current list of dice values.
      category (str): The scoring category (e.g., "ones", "twos", "yahtzee").

  Returns:
      int: The score for the chosen category based on the dice roll.
  """
  score = 0
  if category == "ones":
    score = sum(die for die in dice if die == 1)
  elif category == "twos":
    score = sum(die for die in dice if die == 2)
  # ... (similar logic for other categories)
  elif category == "yahtzee":
    if all(die == dice[0] for die in dice):
      score = 50
  else:
    score = sum(dice)  # Chance category
  return score

In [4]:
def score(dice: list[int]) -> dict[str, int]:
  """
  Calculates the score for each Yahtzee category for a given dice roll.

  Args:
      dice: A list of integers representing the rolled dice values (1-6).

  Returns:
      A dictionary where keys are Yahtzee category names (strings) and values are
      corresponding scores (integers).
  """
  scores = {
      "ones": sum(die == 1 for die in dice),
      "twos": sum(die == 2 for die in dice),
      "threes": sum(die == 3 for die in dice),
      "fours": sum(die == 4 for die in dice),
      "fives": sum(die == 5 for die in dice),
      "sixes": sum(die == 6 for die in dice),
      "three_of_a_kind": sum(dice) if len(set(dice)) <= 3 else 0,
      "four_of_a_kind": sum(dice) if len(set(dice)) <= 2 else 0,
      "full_house": 25 if (len(set(dice)) == 2 and (2 in dice.count(dice[0]) or 3 in dice.count(dice[0]))) else 0,
      "small_straight": 30 if sorted(dice) == [1, 2, 3, 4] or sorted(dice) == [2, 3, 4, 5] else 0,
      "large_straight": 40 if sorted(dice) == [1, 2, 3, 4, 5] or sorted(dice) == [2, 3, 4, 5, 6] else 0,
      "yahtzee": 50 if all(dice.count(val) == 1 for val in dice) else 0,
      "chance": sum(dice),
  }
  return scores

In [5]:
def is_category_available(scores, category):
  """Checks if the chosen scoring category is available.

  Args:
      scores (dict[str, int]): The current Yahtzee scorecard with categories and scores.
      category (str): The scoring category to check for availability.

  Returns:
      bool: True if the category is not yet used (score is None), False otherwise.
  """
  return scores[category] is None

In [6]:
def choose_best_move_simple(dice: list[int], available_scores: dict[str, bool]) -> str:
  """
  Chooses the best move (category) based on the highest fixed score
  among available categories.

  Args:
      dice: A list of integers representing the rolled dice values (1-6).
      available_scores: A dictionary where keys are Yahtzee category names
                         (strings) and values are True/False indicating
                         availability.

  Returns:
      A string representing the Yahtzee category with the highest
      fixed score (e.g., Yahtzee, Full House) among available categories.
  """
  best_score = 0
  best_category = None
  for category, available in available_scores.items():
    if available:
      category_score = calculate_score(dice, category)
      if category_score > best_score and category_score in (50, 40, 25):  # Fixed high-scoring categories
        best_score = category_score
        best_category = category
  return best_category or "re-roll"  # Return "re-roll" if no available category found

In [7]:
def display_scoreboard(scores):
  """Prints the current state of the Yahtzee scorecard.

  Args:
      scores (dict[str, int]): The current Yahtzee scorecard with categories and scores.
  """
  print("Yahtzee Scorecard:")
  for category, score in scores.items():
    print(f"- {category}: {score}")

In [8]:
def get_user_choice(prompt, valid_options):
  """Gets user input with validation based on provided options.

  Args:
      prompt (str): The message to display to the user requesting input.
      valid_options (list[str]): A list of valid options the user can choose from.

  Returns:
      str: The user's chosen option from the list of valid options.
  """
  while True:
    choice = input(prompt)
    if choice in valid_options:
      return choice
    else:
      print("Invalid choice. Please try again.")

In [9]:
def get_re_roll_choices(dice):
  """Prompts user for dice to re-roll and returns a list of choices.

  Args:
      dice (list[int]): The current list of dice values.

  Returns:
      list[str]: A list of strings representing the dice indices the user wants to re-roll.
  """
  valid_choices = [str(i) for i in range(1, len(dice) + 1)]
  return get_user_choice("Enter dice to re-roll (e.g., 1 2 4): ", valid_choices).split()

In [10]:
def get_scoring_category(scores):
  """Prompts user for a scoring category and validates availability."""
  available_categories = [category for category, score in scores.items() if score is None]
  return get_user_choice("Choose a scoring category: ", available_categories)

In [11]:
def show_dice(dice):
  """Displays the current roll of the dice."""
  print("Current roll:", dice)


In [12]:
def show_scores(scores):
  """Displays the current Yahtzee scorecard."""
  display_scoreboard(scores)

In [13]:
def track_score_history(score_history, turn, roll, dice, category, score):
  """Tracks the history of rolls, dice, and scores for analysis."""
  score_history.append({
      "turn": turn,
      "roll": roll,
      "dice": dice,
      "category": category,
      "score": score
  })

In [14]:
import unittest
import random

# Define the roll_dice function (assuming it's not already defined elsewhere)
def roll_dice(num_dice):
  """
  Simulates rolling a specified number of dice.

  Args:
      num_dice: An integer representing the number of dice to roll.

  Returns:
      A list of integers representing the rolled dice values (1-6).
  """
  if num_dice <= 0:
    raise ValueError("Number of dice must be a positive integer.")

  return [random.randint(1, 6) for _ in range(num_dice)]

# Test Cases (adapted as functions for easier execution)
def test_roll_dice_5():
  """Test rolling 5 dice, expect a list of 5 random integers between 1 and 6."""
  rolled_dice = roll_dice(5)
  if len(rolled_dice) != 5:
    print("Error: Incorrect number of dice rolled.")
    return
  for die in rolled_dice:
    if not (1 <= die <= 6):
      print("Error: Die value outside valid range (1-6).")
      return
  print("Test Passed: Roll_dice(5) returned a valid list of 5 dice values.")

def test_roll_dice_0():
  """Test rolling 0 dice, expect a ValueError for invalid input."""
  try:
    roll_dice(0)
  except ValueError as e:
    print("Test Passed: Roll_dice(0) raised a ValueError:", e)
  else:
    print("Error: Roll_dice(0) did not raise a ValueError.")

def test_roll_dice_multiple():
  """Test rolling dice multiple times, verify different results."""
  rolls = [roll_dice(5) for _ in range(3)]
  if rolls[0] == rolls[1]:
    print("Error: Consecutive rolls produced identical results.")
  elif rolls[1] == rolls[2]:
    print("Error: Consecutive rolls produced identical results.")
  else:
    print("Test Passed: Multiple rolls produced different results.")

# Execute Test Cases
test_roll_dice_5()
test_roll_dice_0()
test_roll_dice_multiple()

Test Passed: Roll_dice(5) returned a valid list of 5 dice values.
Test Passed: Roll_dice(0) raised a ValueError: Number of dice must be a positive integer.
Test Passed: Multiple rolls produced different results.
